Since I will be working in an IDE and importing my functions and classes in this notebook, I can use the **autoreload** magic method that automatically reloads my code when I modify it, so I do not have to restart the notebook everytime.

**Notebook contains only the code to test implementation and the implementation itself is in the file.**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [50]:
import pandas as pd
import joblib
import json
from src.model import load_dataset, build_model, tune_model, train_model, \
    test_model
from src.transformers import CategoriesExtractor, GoalAdjustor, TimeTransformer, \
    CountryTransformer
from src.config import MODEL_NAME
from truth.truth import AssertThat
from hypothesis.strategies import text, lists, fixed_dictionaries, from_regex
from hypothesis.extra.pandas import column, data_frames
pd.set_option('display.max_columns', 100)

In [7]:
X_train, y_train = load_dataset("X_train.zip", "y_train.zip")
X_train.head()

,photo,name,blurb,goal,slug,disable_communication,country,currency,currency_symbol,currency_trailing_code,deadline,created_at,launched_at,static_usd_rate,creator,location,category,profile,urls,source_url,friends,is_starred,is_backing,permissions
id,,,,,,,,,,,,,,,,,,,,,,,,
810643898,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",Reverence: A Documentary Short on Branded Yarm...,A documentary exploring the phenomenon of cust...,6000.0,reverence-a-documentary-short-on-custom-yarmulkes,False,US,USD,$,True,1383676790,1378142506,1380217190,1.0,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
407153952,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",YOU Release the NEW Goddamn Electric Bill VINY...,Guarantee yourself one of 250 limited vinyl re...,1500.0,you-release-the-new-goddamn-electric-bill-viny...,False,US,USD,$,True,1284757200,1282000087,1282175023,1.0,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
531190382,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",Lonely Estates Album,Lonely Estates are almost done with their debu...,1000.0,lonely-estates-album,False,US,USD,$,True,1319133283,1315424966,1317318883,1.0,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
1253528325,"{""small"":""https://ksr-ugc.imgix.net/assets/012...",Crit Hit! 2016 - a tabletop roleplaying gather...,A tabletop RPG focused event where you can pla...,3000.0,crit-hit-2016-a-tabletop-roleplaying-gathering-in,False,US,USD,$,True,1460442791,1451709060,1456990391,1.0,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":1,""should_show_fea...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
379783411,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",Hillsboro Arts Preschool Yearbook,"The Yearbook will be a collection of drawings,...",2500.0,hillsboro-arts-preschool-yearbook,False,US,USD,$,True,1356197572,1336233031,1353605572,1.0,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN


In [8]:
X_train.category[379783411]

'{"urls":{"web":{"discover":"http://www.kickstarter.com/discover/categories/publishing/art%20books"}},"color":14867664,"parent_id":18,"name":"Art Books","id":45,"position":3,"slug":"publishing/art books"}'

**Here we will create a new transformer that allows to extra categories from the `category` column that is stored as a json. We want this transformer to have a parameter `validate` that allows us to choose between filtering to use only a subset of hardcoded categories we care about or get all categories found in the json.**

**The `transform` method should return two new columns, `gen_cat` the generic categorie, and `precise_cat` the precise category, those are extracted by assuming the json contains a string in the format `gen_cat/precise_cat`. We provide the method `extract_categories` that gets the two categories, and filter if necessary, so you only have to implement `fit` and `transform` to return the two new columns we want:**

In [9]:
ce = CategoriesExtractor()
ce.fit(X_train)
ce.transform(X_train).head()

,gen_cat,precise_cat
id,,
810643898,film & video,documentary
407153952,music,misc
531190382,music,rock
1253528325,games,misc
379783411,publishing,misc


In [10]:
X_train.columns

Index(['photo', 'name', 'blurb', 'goal', 'slug', 'disable_communication',
       'country', 'currency', 'currency_symbol', 'currency_trailing_code',
       'deadline', 'created_at', 'launched_at', 'static_usd_rate', 'creator',
       'location', 'category', 'profile', 'urls', 'source_url', 'friends',
       'is_starred', 'is_backing', 'permissions'],
      dtype='object')

**Simple transformer that returns column adjusted_goal which is the goal multiplied by static_usd_rate**

In [11]:
ga = GoalAdjustor()
ga.transform(X_train).head()

,adjusted_goal
id,
810643898,6000.0
407153952,1500.0
531190382,1000.0
1253528325,3000.0
379783411,2500.0


### Feature Engineering

**Computes number of days betweeen deadline and launched, launched and created.**

In [23]:
tt = TimeTransformer()
tt.transform(X_train).head()

,launched_to_deadline,created_to_launched
id,,
810643898,40,24
407153952,29,2
531190382,21,21
1253528325,39,61
379783411,30,201


### CountryTransformer
**Returns a larger area for the country feature, allowing to have less dummy features later. Provides a dictionary of countries and their corresponding groups.**

In [16]:
ct = CountryTransformer()
ct.transform(X_train).sample(10)

,country
id,
918458039,UK & Ireland
1875366029,US
1237505203,Europe
1500782856,US
148444406,US
1826807198,US
1509860155,US
499745770,Europe
1808799552,UK & Ireland


## ColumnTransformer & Pipeline

**Here we will implement the `build_model` function that is defined in `model.py`. This function should return a new Pipeline object that has two stages:**
- `preprocessor`: A ColumnTransformer object that has all your preprocessing steps
- `model`: A predictive model, here we will use the `DecisionTreeClassifier`

**We are providing code to build two simple intermediary Pipeline objects: `cat_processor` and `country_processor`. Those are just combining our `CategoriesExtractor` and `CountryTransformer` with a `OneHotEncoder` stage so the output is an array of 1 and 0 for all.**

- the main ColumnTransformer that puts all the transformers together and applies them on the right columns
- the final Pipeline object that puts together the preprocessor and model.

The code below will get a new model using this function, train it on the data and generate predictions

In [17]:
model = build_model()
model.fit(X_train, y_train)
model.predict(X_train)

array([1, 1, 1, ..., 0, 1, 1])

## Steps above allows us to set up everything we need to build the model. Now we can focus on tuning, training and testing.

* Training a model can take a while
* It is useful to be able to save it pre-trained to disk
* Once the model is trained, you can efficiently load use it in production

## Tuning

In [19]:
tune_model()

Best Hyperparameters: {'model__max_depth': 7, 'model__min_samples_split': 5}
Best score: 64.22%


## Training

In [20]:
train_model()

**If this properly saved a model, the code below should load it and generate predictions:**

In [21]:
model_loaded = joblib.load(MODEL_NAME)
model_loaded.predict(X_train)

array([1, 0, 1, ..., 1, 1, 0])

## Testing

In [22]:
test_model()

Accuracy on test set: 65.89%
              precision    recall  f1-score   support

           0       0.71      0.57      0.64       511
           1       0.62      0.75      0.68       474

    accuracy                           0.66       985
   macro avg       0.67      0.66      0.66       985
weighted avg       0.67      0.66      0.66       985



**All the code properly modular and seperated, so it is easy to make modifications. Modify the existing Transformers by changing the way category is being computed. Just need to focus on the transform method inside the class. If there is need to add extra step just create a new Transformer and then test it individually making sure it works fine, and then in the model.py just need to add it in the build_model() as an extra step. Either create a Pipeline object or add it directly to top level column transformer. If I want to change the algorithm I'm using it's also quite simple.**

You can use the `run.py` script to work with your model on the command line:

- `python run.py tune`: will tune your model
- `python run.py train`: will train it
- `python run.py test`: will test it
- `python run.py unittest`: will run tests 

## PyTruth: friendlier assertions

In [25]:
data = X_train.iloc[1]
data

photo                     {"small":"https://ksr-ugc.imgix.net/assets/011...
name                      YOU Release the NEW Goddamn Electric Bill VINY...
blurb                     Guarantee yourself one of 250 limited vinyl re...
goal                                                                   1500
slug                      you-release-the-new-goddamn-electric-bill-viny...
disable_communication                                                 False
country                                                                  US
currency                                                                USD
currency_symbol                                                           $
currency_trailing_code                                                 True
deadline                                                         1284757200
created_at                                                       1282000087
launched_at                                                      1282175023
static_usd_r

In [26]:
AssertThat(data['deadline']).IsNonZero()

In [27]:
AssertThat(data['deadline']).IsZero()

TruthAssertionError: Not true that <1284757200> is zero.

In [29]:
AssertThat(data['blurb']).Contains('music')

TruthAssertionError: <'Guarantee yourself one of 250 limited vinyl records of the new Goddamn Electric Bill album, "Jazz"'> should have contained <'music'>.

In [30]:
AssertThat(data['blurb']).Contains('album')

## Hypothesis

In [33]:
for i in range(0, 10):
    # I want a list of text, please generate bunch of examples.
    print(lists(text()).example())

['\x1b\U000ecabe\x16\x1b(\U00044789\U000f6b45', '\U000985b0𫑲\U00077df3', '槪\x16\U000edb21', '', '\U00061a95𭣎\x14\U0005ec65\U000c0e3f', "\t.\U000b01e9\U00050b70𑠰'(\U0003d043⚜\U000f72ee\x0e\U00041aac\U000dfe63𥅴\U0003a145\x1e\x16\x16\U00065e42!\x1f", '\x1f', '', '+', '', '', '\x0b\U00085681\n\U000aaf5e&', '', '\x0e\x08\U000de946', '\U000523b1𣣠\U00048aaf', '', '\U0002ef16', '\x1f/\x18,\U000397c3\x0c\x11', 'ᦏ(\x1c\U0003f05f\U000fa3fb\x10\U00054931\x0b', '\x18\U00101084\U000d569c\U000c4058\x07\x03\x12']
["\U00092f8f\x04\U0003c48f자\x00*\U000a6925\U000d5eb9#\U000c465e'\U0004c3df\U00067e01\U00061703\x0c\x03\U00049511\U000d3b76\x17", '\U000b4f0c\r']
['沗\x13\U00049049', '&\U00083eb3\x1d', '\x00\x19\U00108715\U0006c7c5\x1b"\U000b959c\x01\U000ced29\x00\x1c\U000b39f8\U000f0d7d\t', '\U00057cdb/\U000e7fc8\U000732f5\U00082a31\x170+\U0005a472\U000d587d', '\x0b!\U000c2b27)\x01𩀞\U000917dc\U00104e72', '\U00086f3e\U0009fd12\U0009f42f\U00067445\x01\U0007419c\U00016c03\x07\U0010a0db\U000e05a0\U0010c5f5\x00\x0

In [35]:
data_frames([column('goal', dtype=float), column('static_usd_rate', dtype=float)]).example()

,goal,static_usd_rate
0,2.220446e-16,-5.000000e-01
1,2.220446e-16,-1.000000e+07
2,2.220446e-16,-5.000000e-01
3,2.220446e-16,-2.000010e+00
4,2.220446e-16,-5.000000e-01
5,2.220446e-16,0.000000e+00
6,2.220446e-16,-5.000000e-01
7,2.220446e-16,1.000000e+07


In [39]:
for i in range(0, 10):
    # generates random nested dictionaires
    print(fixed_dictionaries({'slug': text()}).example())

{'slug': '\U00100ea8\t\U000dc611\x00'}
{'slug': '\U000d0b03#\x06'}
{'slug': '\U000ad2dd'}
{'slug': '%'}
{'slug': '\U000eb31d\x16\rꖕ'}
{'slug': '읎\U000aa5ee䄹\x18'}
{'slug': '%\U0010ac46\U0009cfe4\x1c\x06,\U00088fa6\U000c7c6d\x11\r\U0003bc54\U000b9679'}
{'slug': ''}
{'slug': ''}
{'slug': '꒣䌺\x19'}


In [41]:
from_regex("^[abc]{3}$").example()

'bca'

In [45]:
fixed_dictionaries({'data': from_regex("^[abc]{3}$")}).map(json.dumps).example()

'{"data": "ccc\\n"}'